In [1]:
import transformers

In [2]:
import json
import os
import sys
import argparse

import numpy as np
import pandas as pd
import torch
import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from torch.optim import AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import balanced_accuracy_score

import smdebug.pytorch as smd
from smdebug import modes
from smdebug.pytorch import get_hook

MAX_LEN = 128

#distilbert tokenizer - distilbert uncased
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

[2022-09-11 11:02:03.525 ip-172-16-68-51.ec2.internal:24766 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-09-11 11:02:03.722 ip-172-16-68-51.ec2.internal:24766 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [49]:
from io import StringIO
import unicodedata

In [26]:
MAX_LEN = 128

def prepare_string(string, tokenizer, MAX_LEN):
    
    input_ids = list(tokenizer(string)['input_ids'])
    
    input_ids_padded = []
    while len(input_ids) < MAX_LEN:
        input_ids.append(0)
    
    # attention mask is 0 where length is padded, otherwise it is 1
    
    att_mask = [int(id_ > 0) for id_ in input_ids]
    
    # convert to PyTorch data types.
    test_inputs = torch.tensor(input_ids)
    test_masks = torch.tensor(att_mask)
    
    return {'input_ids': test_inputs, 'attention_mask': test_masks}
    

In [36]:
def remove_accents(input_str: str) -> str:
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    
    return only_ascii.decode()


def string_cleaning(string_series: pd.Series) -> pd.Series:
    
    clean_series = (string_series
                    .astype(str)
                    .str.replace('[^\w\s]',' ', regex=True)
                    .str.replace('\n', ' ')
                    .str.replace(r'[\s+]', ' ', regex=True)
                    .apply(remove_accents))
    
    return clean_series

In [46]:
a = StringIO('{"description": ["this is a string"]}')

In [42]:
a

In [47]:
df = (pd.read_json(a))

In [51]:
b = (df
     .assign(description = lambda df: string_cleaning(df['description'])))

In [52]:
def prepare_string(string, tokenizer, MAX_LEN):
    
    input_ids = list(tokenizer(string)['input_ids'])
    
    input_ids_padded = []
    while len(input_ids) < MAX_LEN:
        input_ids.append(0)
    
    # attention mask is 0 where length is padded, otherwise it is 1
    
    att_mask = [int(id_ > 0) for id_ in input_ids]
    
    # convert to PyTorch data types.
    test_inputs = torch.tensor(input_ids)
    test_masks = torch.tensor(att_mask)
    
    return {'input_ids': test_inputs, 'attention_mask': test_masks}

In [53]:
prepare_string(b.iloc[0,0], tokenizer, MAX_LEN)

{'input_ids': tensor([ 101, 2023, 2003, 1037, 5164,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0

In [4]:
a = 'this is a duck'

StringIO()

In [5]:
tokenizer(a)

{'input_ids': [101, 2023, 2003, 1037, 9457, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}